In [1]:
import csv
def get_domains(examples):
    d = [set() for i in examples[0]]
    #print(d)
    for x in examples:
        for i, xi in enumerate(x):
            d[i].add(xi)
            #print(d[i])
    return [list(sorted(x)) for x in d]

# OUTPUT
 Sunny, Warm, Normal, Strong, Warm, Same,     Y
 Sunny, Warm, High,    Strong, Warm, Same,   Y
 Rainy,  Cold, High,   Strong, Warm, Change, N
 Sunny, Warm,  High,  Strong,   Cool, Change, Y
 
Set ()  	 Set ()   Set ()   Set ()   Set ()   Set ()   Set ()   
  
      Set ()   	    Set ()   	Set ()   	Set ()   	Set ()   	  Set ()   	   Set ()   
[sunny,rainy]  	[warm,cold]	[Normal,high]	[strong]	[Warm,cool]	[same,change]	[Y,N]



In [2]:
def more_general(h1, h2):
    more_general_parts = []
    for x, y in zip(h1, h2):
        mg = x == "?" or (x != "0" and (x == y or y == "0"))
        more_general_parts.append(mg)
        print(more_general_parts)
    return all(more_general_parts)

# output
  more_general(hypothesis, example)
                     h1                                h2
                     x                                  y
     G1    {('?', '?', '?', '?', '?', '?')}      Sunny, Warm, Normal, Strong, Warm, Same, 
     
[True]
[True, True]
[True, True, True]
[True, True, True, True]
[True, True, True, True, True]
[True, True, True, True, True, True]

     S1  { 0,0,0,0,0,0 }                         Sunny, Warm, Normal, Strong, Warm, Same   
[False]
[False, False]
[False, False, False]
[False, False, False, False]
[False, False, False, False, False]
[False, False, False, False, False, False]

In [3]:
def fulfills(example, hypothesis):
# the implementation is the same as for hypotheses:
    return more_general(hypothesis, example)


# OUTPUT
          example                             hypothesis         
 Sunny, Warm, Normal, Strong, Warm, Same,     ?,?,?,?,?,? 
 
 
 return more_general(hypothesis, example)
 
 
         example                             hypothesis         
 Sunny, Warm, Normal, Strong, Warm, Same,     0,0,0,0,0,0 
 
 
 return more_general(hypothesis, example)

In [4]:
def min_generalizations(h, x):
    h_new = list(h)
    print("label1")
    print(h_new)
    for i in range(len(h)):
        if not fulfills(x[i:i+1], h[i:i+1]):
            h_new[i] = '?' if h[i] != '0' else x[i]
            print(h_new[i])
    return [tuple(h_new)]

# output
h  [( 0,0,0,0,0,0)]

x  Sunny, Warm, Normal, Strong, Warm, Same

[False]
 Sunny
[False]
Warm
[False]
Normal
[False]
Strong
[False]
 Warm
[False]
Same

h Sunny, Warm, Normal, Strong, Warm, Same

x  Sunny, Warm, High,    Strong, Warm, Same,

  [True, True, False, True, True, True]
   
  Sunny, Warm,  ?,   Strong, Warm, Same,

In [5]:
def min_specializations(h, domains, x):
    results = []
    for i in range(len(h)):
        if h[i] == "?":
            for val in domains[i]:
                if x[i] != val:
                    h_new = h[:i] + (val,) + h[i+1:]
                    print(h_new)
                    results.append(h_new)
        elif h[i] != "0":
            h_new = h[:i] + ('0',) + h[i+1:]
            results.append(h_new)
    return results

# Output
h=G[2]: {('?', '?', '?', '?', '?', '?')}
domains = [[' Rainy', ' Sunny'], ['Cold', 'Warm'], ['High', 'Normal'], ['Strong'], ['Cool', 'Warm'], ['Change', 'Same']]
x= Rainy,  Cold, High,   Strong, Warm, Change


(' Sunny', '?', '?', '?', '?', '?')
('?', 'Warm', '?', '?', '?', '?')
('?', '?', 'Normal', '?', '?', '?')
('?', '?', '?', '?', 'Cool', '?')
('?', '?', '?', '?', '?', 'Same')

 

In [6]:
def generalize_S(x, G, S):
    S_prev = list(S)
    print("label2")
    print(S_prev)
    for s in S_prev:
        
        if not fulfills(x, s):
            print("label3")
            print(s)
            S.remove(s)
            Splus = min_generalizations(s, x)
            print(Splus)
            print("label4")
## keep only generalizations that have a counterpart in G
            S.update([h for h in Splus if any([more_general(g,h)
            for g in G])])
            print(S)
            print("label5")

    return S

# Output
S.update([h for h in Splus if any([more_general(g,h)
            for g in G])])
            
It updates the contents of Splus in S            

In [7]:
def specialize_G(x, domains, G, S):
    G_prev = list(G)
    for g in G_prev:
       
        if fulfills(x, g):
            G.remove(g)
            Gminus = min_specializations(g, domains, x)
## keep only specializations that have a conuterpart in S
            G.update([h for h in Gminus if any([more_general(h, s)
            for s in S])])

    return G

# Negative hypothesis

x= Rainy,  Cold, High,   Strong, Warm, Change
domains = [[' Rainy', ' Sunny'], ['Cold', 'Warm'], ['High', 'Normal'], ['Strong'], ['Cool', 'Warm'], ['Change', 'Same']]
S[3]: {(' Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same')}
G[2]: {('?', '?', '?', '?', '?', '?')}

 Gminus = min_specializations(g, domains, x)

G.update([h for h in Gminus if any([more_general(h, s)for s in S])])

                    h                                                        S
(' Sunny', '?', '?', '?', '?', '?')                           S[3]: {(' Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same')}
('?', 'Warm', '?', '?', '?', '?')
('?', '?', 'Normal', '?', '?', '?')  --discards
('?', '?', '?', '?', 'Cool', '?')    --discards
('?', '?', '?', '?', '?', 'Same')    

It retains only
G[3] : (' Sunny', '?', '?', '?', '?', '?')  ('?', 'Warm', '?', '?', '?', '?') ('?', '?', '?', '?', '?', 'Same')    




In [8]:
def candidate_elimination(examples):
    domains = get_domains(examples)[:-1]
    print(domains)
    n = len(domains)
    print(n)
    G = set([("?",)*n])
    print(G)
    S = set([("0",)*n])
    print(S)
    print("Maximally specific hypotheses - S ")
    print("Maximally general hypotheses - G ")
    i=0
    print("\nS[0]:",str(S),"\nG[0]:",str(G))
    for xcx in examples:
        i=i+1
        x, cx = xcx[:-1], xcx[-1] # Splitting data into attributes and decisions
        if cx=='Yes': # x is positive example
            G = {g for g in G if fulfills(x, g)}
            print(G)
            S = generalize_S(x, G, S)
            print(S)
        else: # x is negative example
            S = {s for s in S if not fulfills(x, s)}
            G = specialize_G(x, domains, G, S)
        print("\nS[{0}]:".format(i),S)
        print("G[{0}]:".format(i),G)
    return
with open('neg.csv') as csvFile:
    examples = [tuple(line) for line in csv.reader(csvFile)]
    print(examples)
candidate_elimination(examples)

[('cloudy', 'cold', 'high', 'strong', 'warm', 'change', 'No'), ('sunny', 'warm', 'high', 'strong', 'warm', 'same', 'Yes'), ('sunny', 'warm', 'high', 'strong', 'cool', 'change', 'Yes'), ('sunny', 'warm', 'normal', 'strong', 'warm', 'same', 'Yes')]
[['cloudy', 'sunny'], ['cold', 'warm'], ['high', 'normal'], ['strong'], ['cool', 'warm'], ['change', 'same']]
6
{('?', '?', '?', '?', '?', '?')}
{('0', '0', '0', '0', '0', '0')}
Maximally specific hypotheses - S 
Maximally general hypotheses - G 

S[0]: {('0', '0', '0', '0', '0', '0')} 
G[0]: {('?', '?', '?', '?', '?', '?')}
[False]
[False, False]
[False, False, False]
[False, False, False, False]
[False, False, False, False, False]
[False, False, False, False, False, False]
[True]
[True, True]
[True, True, True]
[True, True, True, True]
[True, True, True, True, True]
[True, True, True, True, True, True]
('sunny', '?', '?', '?', '?', '?')
('?', 'warm', '?', '?', '?', '?')
('?', '?', 'normal', '?', '?', '?')
('?', '?', '?', '?', 'cool', '?')
('